In [9]:
import numpy as np
import sys
import os
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from bbox_toolbox import poly2bbox
import csv
from tqdm import tqdm
import json
import pandas as pd
def darknet_format_anno(src,save_location,threshold=0.005):
    classes = {}
    abandon_class = {}
    with open(src,"r+") as f:
        annos = f.readlines()
    os.makedirs(save_location,exist_ok=True)
    for anno in tqdm(annos):
        anno = json.loads(anno)
        img_id = anno["image_id"]
        ignored = anno["ignore"]
        file_name = anno["file_name"]
        bboxes = []
        fname = os.path.join(save_location,img_id) + ".txt"

        for text in anno["annotations"]:
            for character in text:
                c_true = character["is_chinese"]
                x_c,y_c,w,h = poly2bbox(character["polygon"], 2048)
                
                if x_c > 1 or y_c > 1 or w < threshold or h < threshold:
                    continue
                if not c_true:
                    continue
                else:
                    cls = 0
                    bboxes.append([cls,x_c,y_c,w,h])
        with open(fname,"w",newline="") as f:
            w = csv.writer(f,delimiter=' ')
            for b in bboxes:
                w.writerow(b)

def darknet_format_txt(src,img_location,txt_name):
    
    files = os.listdir(img_location)
    imgs = []
    classes = []
    with open(src,"r+") as f:
        annos = f.readlines()

    for anno in tqdm(annos):
        anno = json.loads(anno)
        img_id = anno["image_id"]
        ignored = anno["ignore"]
        file_name = anno["file_name"]
        imgs.append(file_name)
    qualified = [x for x in files if x in imgs]
    qualified = [os.path.join(img_location,x) for x in qualified]

    with open(txt_name,"a+") as f:
        for line in qualified:
            f.write(line+"\n")
            

def draw_bbox_test(img,bbox):
    fig,ax = plt.subplots(1,figsize=(15,15))
    ax.imshow(p)
    x,y,w,h = bbox
    rect = patches.Rectangle((x*2048,y*2048),w*2048,h*2048,linewidth=1,edgecolor='r',facecolor='none')
    ax.add_patch(rect)
    plt.show()

In [8]:
train_src = "annotations/train.jsonl" 
valid_src = "annotations/val.jsonl"
train_bbox_save = "/home/jovyan/EJ/ctw_data/trainval/labels"
train_img = "/home/jovyan/EJ/ctw_data/trainval/images"
val_image = "/home/jovyan/EJ/ctw_data/trainval/images"
train_text = "ctw_train.txt"
val_text = "ctw_val.txt"
threshold = 0.008

darknet_format_anno(train_src,train_bbox_save)
darknet_format_anno(valid_src,train_bbox_save)
darknet_format_txt(train_src,train_img,train_text)
darknet_format_txt(valid_src,val_image,val_text)

100%|██████████| 1597/1597 [00:00<00:00, 4497.49it/s]


In [5]:
# train_src = "annotations/test_cls.jsonl" 
# train_bbox_save = "/home/jovyan/EJ/ctw_data/test/labels"
# train_img = "/home/jovyan/EJ/ctw_data/test/images"
# test_text = "ctw_test.txt"
# threshold = 0.008

# darknet_format_anno(train_src,train_bbox_save)

# darknet_format_txt(train_src,train_img,train_text)


True
